In [1]:
import pandas as pd

In [15]:
AOIS = ['Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4',
       'Paragraph5', 'Paragraph6', 'Text_Area', 'SubFigure1', 'SubFigure2',
       'SubFigure3', 'Figure', 'Formula']

In [16]:
# Read gaze data with events and AOIs using pandas library
data = pd.read_csv("data/gazeDataWithAOIsAndEvents.csv")
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [17]:
# Preview data
data.head()

,Row,Timestamp,EventSource,SlideEvent,StimType,Duration,CollectionPhase,SourceStimuliName,EventSource.1,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,ET_PupilLeft,GazeAccelerationAngle,ET_PupilRight,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,128.9448,NaN,NaN,NaN,NaN,NaN,img,1,141.658,0.5989,0.5257,0.4298,0.5191,0,0,960.5,494.5,0,0,0,1,0,0,1,0,0,0,0,0,960.5,494.5,567.9688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,137.2724,NaN,NaN,NaN,NaN,NaN,img,1,149.984,0.5991,0.5255,0.4300,0.5190,0,0,952.0,509.5,0,0,0,1,0,0,1,0,0,0,0,0,952.0,500.0,567.9048,16.1935,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,145.6203,NaN,NaN,NaN,NaN,NaN,img,1,158.333,0.5990,0.5253,NaN,NaN,0,4,938.0,500.0,1,0,0,0,0,0,1,0,0,0,0,0,948.0,500.0,566.2654,4.7572,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,2.6143,-1369.9611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,153.9470,NaN,NaN,NaN,NaN,NaN,img,1,166.660,0.5989,0.5252,0.4299,0.5187,0,0,948.0,499.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,500.0,567.7859,27.6866,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,2753.7102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,162.2703,NaN,NaN,NaN,NaN,NaN,img,1,174.983,0.5991,0.5251,0.4301,0.5187,0,0,969.5,515.0,0,0,0,1,0,0,1,0,0,0,0,0,969.5,509.0,567.7856,27.6456,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,-4.9222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Derive a dataframe with fixation data only
# Keep only relevant colomns
fixationData = data[['FixID',
       'Fixation Duration']+AOIS].copy(deep=True)

# Since the same fixation is assigned to multiple gazes, it is necessary to drop duplicates
fixationData = fixationData.drop_duplicates()
# Then, it is necessary columns with no FixID
fixationData = fixationData.dropna(subset=['FixID'])


# Reset the index of the dataframe
fixationData = fixationData.reset_index()

In [30]:
# Preview fixation data
fixationData.head()

,index,FixID,Fixation Duration,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
0,1,1.0,224.9734,0,0,0,1,0,0,1,0,0,0,0,0
1,2,1.0,224.9734,1,0,0,0,0,0,1,0,0,0,0,0
2,8,1.0,224.9734,0,0,0,0,0,0,0,0,0,0,0,0
3,53,2.0,141.6935,0,0,0,0,0,0,0,0,0,0,0,0
4,54,2.0,141.6935,1,0,0,0,0,0,1,0,0,0,0,0


In [59]:
# 1. Fixation-based measures
allStats = None

# Iterate through the different AOIS
for aoi in AOIS:
    # For each AOI compute 'sum','mean','max','min' of 'Fixation Duration' 
    stats = fixationData[fixationData[aoi]==1].groupby(aoi).agg({'Fixation Duration':['sum','mean','max','min']})
    
    # Flatten hierarchically indexed pandas.DataFrame from groupby and multiple aggregation  
    stats.columns= ['_'.join(x) for x in list(zip(stats.columns.get_level_values(0), stats.columns.get_level_values(1)))]
    stats = stats.reset_index()
    
    # rename column aoi to "AOI" and set its value to aoi     
    stats.rename(columns={aoi:"AOI"}, inplace=True)
    stats["AOI"] = aoi
    
    #concate
    if allStats is not None:
        allStats = pd.concat([allStats, stats], axis=0)
    else:
        allStats = stats


In [60]:
allStats

,AOI,Fixation Duration_sum,Fixation Duration_mean,Fixation Duration_max,Fixation Duration_min
0,Paragraph1,28863.3645,218.661852,983.2385,66.6587
0,Paragraph2,64400.6573,209.774128,1249.8835,66.6282
0,Paragraph3,9490.3832,206.312678,449.9495,66.6328
0,Paragraph4,7291.0477,251.415438,958.2344,66.6725
0,Paragraph5,7482.3194,174.007428,983.2385,66.6587
0,Paragraph6,3866.0915,161.087146,416.6124,66.6328
0,Text_Area,143033.5649,209.113399,1249.8835,66.6282
0,SubFigure1,17289.2734,227.490439,916.5141,66.6282
0,SubFigure2,33388.2846,216.807043,1124.8921,66.6423
0,SubFigure3,16039.6623,203.033700,1249.8835,66.6352
